In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from answer_judge.vqaEval import VQAEval
from utils.seed import fix_seed

In [2]:
fix_seed(0)
data_reader=ReadData('/root/autodl-tmp/hallscope/VQAIDK/train_qwen')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [ ]:
df.head()

,img_path,question,answer,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Given the question 'What type of trendy jeans...,[answerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,"[Given the question 'What does the bus say?', ...",[unanswerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Given the question 'What pattern is on the wo...,[unanswerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Given the question 'Is the dog sitting under ...,[answerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Given the question 'How many bikes are in the...,[answerable],{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None


In [ ]:
df.shape

(10000, 5)

In [6]:
i=300

In [ ]:
df.iloc[i]['img_path'],df.iloc[i]['most_likely']['response'],df.iloc[i]['question'],df.iloc[i]['answer']

(['/root/autodl-fs/coco_images/train/COCO_train2014_000000414027.jpg'],
 ['Unanswerable'],
 ["Given the question 'Who owns the photo?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'. \n"],
 ['unanswerable'])

In [8]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [9]:
embeddings=np.concatenate(embeddings,axis=0)

In [10]:
embeddings.shape

(10000, 29, 3584)

In [11]:
df['answer']=df['answer'].apply(lambda x: x[0])

In [12]:
gt_label=df['answer'].to_list()
gt_label

['answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'answerable',
 'answerable',
 'answerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'answerable',
 'answerable',
 'unanswerable',
 'answerable',
 'answerable',
 'unanswerable',
 'answerable',
 'answerable',
 'answerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'answerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'answerable',
 'unanswerable',
 'unanswerable',
 'unanswerable',
 'an

In [13]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [14]:
rsps[0]

'Answerable'

In [15]:
is_correct=[]
for i,x in enumerate(gt_label):
    if rsps[i].lower()==x.lower():
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [16]:
is_correct.shape

(10000,)

In [ ]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 10000 responses. 6177 answer correctly and 3823 answer wrongly


In [18]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 4000
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:4000
validset length:4000
testset length:2000
trainset postive: 2437 negative: 1563
valset postive: 2474 negative: 1526
testset postive: 1266 negative: 734


In [19]:
all_embeddings = embeddings
all_embeddings.shape

(10000, 29, 3584)

In [20]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [21]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Train Linear Prob to Get Better Results

In [22]:
best_layer=20

In [24]:
X_train=embedding_train[:, best_layer,:]
y_train=hall_label_wild

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [25]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [26]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.7915 f1:0.830142566191446 auroc:0.8563289082307768  b-acc:0.7866448424740973 auc-pr:0.8923815832914683
